In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# Decision trees
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
#import graphviz
import os

In [27]:
# Set up the data that we want to create a DT on.
# import the fx data , econ and value data for EURUSD.
# then create the features (on trend and econ data) standardise and run a DT on the x_train sample.
# what is  target? 1 day ahead or long days ahead? trade on binary data.
csv_file = {"FXData" : r"C:\Users\edgil\Documents\Masters\dissertation\data\CurrencyData.csv",
            "ValueData" : r"",
            "EconData" : r"",
            }
fxdata = pd.read_csv(csv_file["FXData"])

In [29]:
# use this to make sure the dates are imported correctly as UK dates
fxdata['Date'] = pd.to_datetime(fxdata['Date'], format= '%d/%m/%Y %H:%M')
# Separate out the EURUSD factor
eurusd = fxdata[["Date", "EURUSD"]]


In [75]:
# Build out the featur set on price, this may need to be created using funcional process.
eurusd["logret"] = np.log(eurusd["EURUSD"]) - np.log(eurusd["EURUSD"].shift(1))
# Standardising the daily rets and accumulating the standardised returns, or should we sum the % ret and standardise by its own history
# is difference between different accumulated retusn horizons the same as the macd?
# should we standardise by the 1 year forward vol?
targetlkbk = 5
lkbk1M = 22
lkbk3M = 66
lkbk6M = 132
lkbk9M = 198
eurusd['1MRet'] = eurusd["logret"].rolling(lkbk1M).sum()
eurusd['3MRet'] = eurusd["logret"].rolling(lkbk3M).sum()
eurusd['6MRet'] = eurusd["logret"].rolling(lkbk6M).sum()
eurusd['9MRet'] = eurusd["logret"].rolling(lkbk9M).sum()
eurusd['1Mv3MRet'] = eurusd["1MRet"].sub(eurusd["3MRet"])
eurusd['1Mv6MRet'] = eurusd["1MRet"].sub(eurusd["6MRet"])
# create a target vector to train on.
# must think deeply about what this will look like
# to start, this is a 1M forard return calculation, is it right to use overlapping 1M fwd rets? it seems not...
eurusd["target"] = eurusd['logret'].iloc[::-1].rolling(targetlkbk).sum().values[::-1]
eurusd['target_binary'] = eurusd['target'].apply(np.sign) 

In [76]:
# Very important step is to truncate the data so that we do not see the last 1 year of data.
# Q. should we not have a rollign window type of model? or just always aggregate the data from the start?
# how ong is testing? we should we the train and test to sizes which make sense to the type of model we use going forward.
eurusd = eurusd.loc[eurusd['Date'] < "2018-01-01 00:00"]
# train size should be at least 5 years?
eurusd_train = eurusd.loc[eurusd['Date'] < "2010-01-01 00:00"]
eurusd_test = eurusd.loc[eurusd['Date'] > "2010-02-01 00:00"]
# create a target vector to train on.
# must think deeply about what this will look like



In [83]:
print(eurusd_train_normed.head(50))
# Standardise the data using sklearn
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
# standardise the data now.
# remove dates
data_cols = [col for col in eurusd_train.columns if col not in ["Date", "target", "target_binary"]]
data_train = eurusd_train.loc[:, data_cols]
data_test = eurusd_test.loc[:, data_cols]
eurusd_train_normed = pd.DataFrame(scaler.fit_transform(data_train), columns = list(data_train.columns))
eurusd_test_normed = pd.DataFrame(scaler.transform(data_test), columns = list(data_test.columns)) 

      EURUSD    logret     1MRet  3MRet  6MRet  9MRet  1Mv3MRet  1Mv6MRet
0  -0.857575       NaN       NaN    NaN    NaN    NaN       NaN       NaN
1  -0.857575 -0.004084       NaN    NaN    NaN    NaN       NaN       NaN
2  -0.834812  3.374106       NaN    NaN    NaN    NaN       NaN       NaN
3  -0.834812 -0.004084       NaN    NaN    NaN    NaN       NaN       NaN
4  -0.838484 -0.547908       NaN    NaN    NaN    NaN       NaN       NaN
5  -0.838484 -0.004084       NaN    NaN    NaN    NaN       NaN       NaN
6  -0.837750  0.104713       NaN    NaN    NaN    NaN       NaN       NaN
7  -0.837750 -0.004084       NaN    NaN    NaN    NaN       NaN       NaN
8  -0.853169 -2.292195       NaN    NaN    NaN    NaN       NaN       NaN
9  -0.853169 -0.004084       NaN    NaN    NaN    NaN       NaN       NaN
10 -0.863449 -1.533444       NaN    NaN    NaN    NaN       NaN       NaN
11 -0.863449 -0.004084       NaN    NaN    NaN    NaN       NaN       NaN
12 -0.879359 -2.377221       NaN    Na

In [84]:
#### This is where we choose and set up the models!
# need to re do the models here.
X = eurusd_train_normed
Y = eurusd_train["target_binary"]
X_test = eurusd_test_normed
Y_test = eurusd_test["target_binary"]
# clean the data and nan values
X = X.replace(np.nan, 0)
Y = Y.replace(np.nan, 0)
X_test = X_test.replace(np.nan, 0)
Y_test = Y_test.replace(np.nan, 0)
RF = RandomForestClassifier(n_estimators = 150, max_features = 5)
RF.fit(X, Y)
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X, Y)

In [86]:
clf.feature_importances_

array([0.13750854, 0.20331236, 0.10579135, 0.10831787, 0.10659457,
       0.12786601, 0.10707631, 0.10353298])

In [81]:
# Rough work
#eurusd = eurusd.replace(np.nan, 0)
#print(eurusd.tail(25))
#eurusd["target"] = eurusd['logret'].iloc[::-1].rolling(2).sum().values[::-1]
#print(eurusd.head(10))
data_cols = [col for col in eurusd_train.columns if col not in ["Date", "target", "target_binary"]]
print(data_cols)

['EURUSD', 'logret', '1MRet', '3MRet', '6MRet', '9MRet', '1Mv3MRet', '1Mv6MRet']
